In [19]:
from sklearn.metrics import cohen_kappa_score
import pandas as pd
import json
import os
from concurrent.futures import ThreadPoolExecutor
import numpy as np
import csv
import glob

In [8]:
relabeled_subjects_path = '/dhc/groups/ocdetect/relabeled_subjects'
extracted_labels_path = '/dhc/groups/ocdetect/thesis/extracted_labels'
path_windows = '/dhc/groups/ocdetect/thesis/label_windows/'
clean_labels_path = '/dhc/groups/ocdetect/thesis/cleaned_labels'
clean_windows_path = '/dhc/groups/ocdetect/thesis/cleaned_windows'
main_dir= '/dhc/groups/ocdetect'
thesis = 'thesis'
cleaned_label_windows = 'label_windows_clean'
extracted_labels = 'extracted_labels'
subjects = ["01", "03", "04", "18", "20", "30"]
annotators = ["a1", "a2", "a3", "a4"]

In [16]:
# CHECK IF THIS IS STILL NEEDED!!!!
#Delete all user-set label windows where all annotators didn't set a label
# handles extra labels by several clicks on the watch after a single handwash
# handles windows with no hand washing, when he watch was toughed only after the hw to set a label, but no actual data ho hw is available
def clean_labels(windows_path, subject):
    cleaned_file_name = f'subject_{subject}.csv'
    cleaned_dir_path = os.path.join(main_dir, cleaned_label_windows)
    if not os.path.exists(cleaned_dir_path):
        os.makedirs(cleaned_dir_path)
    output_csv_path = os.path.join(cleaned_dir_path, cleaned_file_name)
    with open(output_csv_path, 'w', newline='') as output_file:
        with open(windows_path, 'r') as windows_file:
            windows_reader = csv.DictReader(windows_file)
            fieldnames = windows_reader.fieldnames
            writer = csv.DictWriter(output_file, fieldnames=fieldnames)
            writer.writeheader()
            for window_row in windows_reader:
                window_filename = window_row['file name']
                window_start = pd.to_datetime(window_row['start'])
                window_end = pd.to_datetime(window_row['end'])
                pairs = []
                for annotator in annotators:
                    path = os.path.join(relabeled_subjects_path, f"{annotator}_subject_{subject}.csv")
                    if os.path.exists(path):
                        pairs.append({'Subject': subject, 'Annotator': annotator})

                flags = [False] * len(pairs)
                for index, pair in enumerate(pairs):
                    subject = pair['Subject']
                    annotator = pair['Annotator']
                    file_path = os.path.join(relabeled_subjects_path, f"{annotator}_subject_{subject}.csv")
                    with open(file_path, 'r') as annotation_file:
                        annotation = csv.DictReader(annotation_file)
                        for label_row in annotation:
                            if window_filename in label_row['datetime']:
                                labels_json = label_row["label"]
                                if not pd.isna(labels_json) and labels_json.strip():
                                    labels = json.loads(labels_json)
                                    for label in labels:
                                        if pd.to_datetime(label['start']) >= window_start and pd.to_datetime(label['end']) <= window_end:
                                            flags[index] = True
                                            break
                if any(flags):
                  writer.writerow(window_row)  
                

                                        
for subject in subjects:
    path = f'/dhc/groups/ocdetect/label_windows/subject_{subject}.csv'
    if os.path.exists(path):
        clean_labels(path, subject)

In [47]:
# checks if there are labels set by annotator that exceed the label set by user
def get_exceeding_labels():
    total_labels = "exceeding_labels2.csv"
    exceeding_durations = []
    with open(total_labels, 'w', newline='') as csvfile:
        fieldnames = ['Subject', 'Annotator', 'File name', 'Duration from start', 'Duration from end', 'HW duration']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
# Iterate through each folder in the main folder
        for file_name in os.listdir(path_windows):
            file_path = os.path.join(path_windows, file_name)
            
            if os.path.isfile(file_path):
                print(f"Reading file: {file_name}")
            
            # Perform file reading operations here
                windows = pd.read_csv(file_path)
                index = file_name.find('_')
                subject = file_name[index+1:index+3]
                for annotator in annotators:
                    path = os.path.join(extracted_labels_path, f"{annotator}_subject_{subject}.csv")
                    if os.path.exists(path):
                        annotationDf = pd.read_csv(path)    
                        for index, row in windows.iterrows():
                            recording = row['file name']
                            start_time_w = pd.to_datetime(row['start'])
                            end_time_w = pd.to_datetime(row['end'])
                            
                            for idx, row2 in annotationDf.iterrows():
                                if recording in row2['datetime']: 
                                    labels_json = row2.get("label")
                                    if not pd.isna(labels_json):
                                        labels = json.loads(labels_json)
                                        for label in labels:
                                            start_time = pd.to_datetime(label['start'], format='%Y-%m-%d %H:%M:%S.%f')
                                            end_time = pd.to_datetime(label['end'], format='%Y-%m-%d %H:%M:%S.%f')
                                            if start_time < end_time_w and end_time > end_time_w:
                                                print("window", start_time_w, " : ", end_time_w)
                                                print("label", start_time, " : ", end_time)
                                                print("----")
                                                #duration = end_time - end_time_w
                                                duration_a = (end_time_w - start_time).total_seconds() 
                                                duration_b = (end_time - end_time_w).total_seconds() 
                                                hw_duration = (end_time - start_time).total_seconds() 
                                                #filter = 
                                                if duration_b > 5 and duration_a > 5:
                                                    exceeding_durations.append(duration_a)
                                                    writer.writerow({'Subject': subject, 'Annotator': annotator, 'File name': recording, 'Duration from start': duration_a, 'Duration from end': duration_b, 'HW duration': hw_duration})
        

get_exceeding_labels()

Reading file: subject_18.csv
window 2022-06-17 13:19:13.580000  :  2022-06-17 13:24:13.580000
label 2022-06-17 13:23:40.960000  :  2022-06-17 13:24:29.500000
----
window 2022-06-18 13:18:07.160000  :  2022-06-18 13:23:07.160000
label 2022-06-18 13:22:36.820000  :  2022-06-18 13:23:10.960000
----
window 2022-06-18 12:44:49.480000  :  2022-06-18 12:45:49.400000
label 2022-06-18 12:45:22.920000  :  2022-06-18 12:45:51.280000
----
window 2022-06-21 12:49:26.600000  :  2022-06-21 12:54:26.600000
label 2022-06-21 12:52:33.700000  :  2022-06-21 12:54:28.200000
----
window 2022-06-21 11:12:41.900000  :  2022-06-21 11:17:41.900000
label 2022-06-21 11:16:08.400000  :  2022-06-21 11:17:43.300000
----
window 2022-06-21 08:19:22.820000  :  2022-06-21 08:24:22.820000
label 2022-06-21 08:23:49.220000  :  2022-06-21 08:24:25.140000
----
window 2022-06-23 07:18:35.260000  :  2022-06-23 07:23:35.260000
label 2022-06-23 07:23:06.380000  :  2022-06-23 07:23:38.120000
----
window 2022-06-25 09:39:33.940000

In [5]:
'''def rename_column_in_csv():
     for file_name in os.listdir(path_windows):
        file_path = os.path.join(path_windows, file_name)
    # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
    
    # Rename the column from 'file name' to 'filename'
        df.rename(columns={'file name': 'filename'}, inplace=True)
    
    # Save the DataFrame back to the same CSV file
        df.to_csv(file_path, index=False)
rename_column_in_csv()'''

In [12]:
# get all file names where a labels is found spreading to two windows, when more than 5 seconds and in each window
def get_exceeding_labels():
    data = []
    for file_name in os.listdir(path_windows):
        file_path = os.path.join(path_windows, file_name)
        
        if os.path.isfile(file_path):
            windows = pd.read_csv(file_path)
            index = file_name.find('_')
            subject = file_name[index+1:index+3]
            for annotator in annotators:
                path = os.path.join(extracted_labels_path, f"{annotator}_subject_{subject}.csv")
                if os.path.exists(path):
                    annotationDf = pd.read_csv(path)
                    for index, row in windows.iterrows():
                        recording = row['filename']
                        start_time_w = pd.to_datetime(row['start'])
                        end_time_w = pd.to_datetime(row['end'])
                        filtered_annotation = annotationDf[annotationDf['filename'].str.contains(recording)]
                        for idx, annotation_row in filtered_annotation.iterrows():
                            start_time = pd.to_datetime(annotation_row['start'], format='%Y-%m-%d %H:%M:%S.%f')
                            end_time = pd.to_datetime(annotation_row['end'], format='%Y-%m-%d %H:%M:%S.%f')
                            if start_time < end_time_w and end_time > end_time_w:
                                duration_a = (end_time_w - start_time).total_seconds() 
                                duration_b = (end_time - end_time_w).total_seconds() 
                                if duration_b > 5 and duration_a > 5:
                                    data.append({
                                        'subject': subject,
                                        'annotator': annotator,
                                        'filename': recording
                                    })

    df_exceeding = pd.DataFrame(data)
    return df_exceeding

In [20]:
#clean labels and windows. in labels delete those than shorter than 5 seconds, delete file and windows that have labels exceeding the end user-set label 

def clean_exceeding_labels():
    files_to_exclude = get_exceeding_labels()
    # clean labels
    for subject in subjects:
        for annotator in annotators:
            filename = f"{annotator}_subject_{subject}.csv"
            path = os.path.join(extracted_labels_path, filename)
            if os.path.exists(path):
                df = pd.read_csv(path)  
                df['start'] = pd.to_datetime(df['start'], format='%Y-%m-%d %H:%M:%S.%f')
                df['end'] = pd.to_datetime(df['end'], format='%Y-%m-%d %H:%M:%S.%f')
                df['duration'] = (df['end'] - df['start']).dt.total_seconds()

                # Filter rows with duration >= 5 seconds
                df_filtered = df[df['duration'] >= 5]
                # Filter out rows with filenames present in files_to_exclude
                df_filtered = df_filtered[~df_filtered['filename'].isin(files_to_exclude['filename'])]
                df_filtered = df_filtered.drop(columns=['duration'])
                out_path = os.path.join(clean_labels_path, filename)
                df_filtered.to_csv(out_path)

    # clean windows
    all_files = glob.glob(os.path.join(path_windows, "*.csv"))
    for file in all_files:
        df = pd.read_csv(file)
        # Filter out rows with filenames present in files_to_exclude
        df_filtered = df[~df['filename'].isin(files_to_exclude['filename'])]
        file_name = os.path.basename(file)
        output_file = os.path.join(clean_windows_path, file_name)
        df_filtered.to_csv(output_file, index=False)


clean_exceeding_labels()


Reading file: subject_18.csv
Reading file: subject_01.csv
Reading file: subject_30.csv
Reading file: subject_20.csv
Reading file: subject_03.csv
Reading file: subject_04.csv


In [53]:
#temp number of hw less than 5 seconds
for subject in subjects:
    for annotator in annotators:
        count = 0
        path = os.path.join(clean_labels_path, f"{annotator}_subject_{subject}.csv")         
        if os.path.exists(path):
            annotationDf = pd.read_csv(path)
            
            for index, row in annotationDf.iterrows():
                start = pd.to_datetime(row['start'])
                end = pd.to_datetime(row['end'])
                duration = (end - start).total_seconds()
                if duration < 5:
                    print(duration)
                    count += 1
            print("subject ", subject, "annotator ", annotator, "number" , count)

4.24
4.14
3.9
3.8
3.04
3.92
3.64
3.24
1.74
2.16
3.94
1.4
3.06
1.8599999999999999
4.06
3.02
4.9399999999999995
2.08
4.1
subject  01 annotator  a1 number 19
subject  01 annotator  a4 number 0
subject  03 annotator  a1 number 0
subject  03 annotator  a2 number 0
subject  04 annotator  a2 number 0
4.82
4.54
1.58
3.2800000000000002
subject  04 annotator  a3 number 4
4.82
subject  18 annotator  a1 number 1
3.68
4.48
3.84
4.52
3.74
subject  18 annotator  a3 number 5
4.68
3.54
3.0
subject  20 annotator  a3 number 3
subject  20 annotator  a4 number 0
subject  30 annotator  a2 number 0
subject  30 annotator  a4 number 0


In [8]:
# this method creates new labels csv files with columns: filename, start, end, instant, label_type
# one rows for one label
# done for easier processing
def process_labels_to_easier_format(out_path):
    
    
    for subject in subjects:
        for annotator in annotators:
            path = os.path.join(relabeled_subjects_path, f"{annotator}_subject_{subject}.csv")
            if os.path.exists(path):
                df = pd.read_csv(path)
                output_file_path = os.path.join(out_path, f"{annotator}_subject_{subject}.csv")
                with open(output_file_path, 'w', newline='') as csvfile:
                    fieldnames = ['filename', 'start', 'end', 'instant', 'timeserieslabels']
                    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                    writer.writeheader()
                    for index, row in df.iterrows():
                        labels_json = row["label"]
                        if not pd.isna(labels_json):
                            labels = json.loads(labels_json)
                            for label in labels:
                                filename = row['datetime']
                                index_ocdetect = filename.find("OCDetect")
                                new_name = filename[index_ocdetect:]
                                start = pd.to_datetime(label['start'], format='%Y-%m-%d %H:%M:%S.%f')
                                end = pd.to_datetime(label['end'], format='%Y-%m-%d %H:%M:%S.%f')
                                instant = label['instant']
                                timeserieslabels  = label['timeserieslabels']
                                writer.writerow({'filename': new_name, 'start': start.strftime('%Y-%m-%d %H:%M:%S.%f'), 'end': end.strftime('%Y-%m-%d %H:%M:%S.%f'), 'instant': instant, 'timeserieslabels': timeserieslabels})

# create a subfolder in thesis folder to save extracted labels
out_path = os.path.join(main_dir, thesis, extracted_labels)
if not os.path.exists(out_path):
    os.makedirs(out_path)
process_labels_to_easier_format(out_path)

In [17]:
#check if there are windows with two labels
path_window = '/dhc/groups/ocdetect/thesis/label_windows'

for subject in subjects:
    path_w = os.path.join(path_window, f"subject_{subject}.csv") 
    windows = pd.read_csv(path_w)
    for annotator in annotators:
        path = os.path.join(extracted_labels_path, f"{annotator}_subject_{subject}.csv")
        if os.path.exists(path):
            df = pd.read_csv(path)
            for index, row in windows.iterrows():
                window_start = row['start']
                window_end = row['end']
                    
                # Find the number of intersecting lines in the annotator file
                intersections = df[
                    (df['start'] < window_end) & 
                    (df['end'] > window_start) & (row['file name'] == df['filename'])
                ]
                    
                    # Check if there is exactly one intersection
                if len(intersections) != 1 and len(intersections) != 0:
                    print(f"Subject {subject}, Window {row['file name']} has {len(intersections)} intersections in {annotator}'s file")

Subject 01, Window OCDetect_01_recording_34_1fd62d17-5957-42bf-b0f3-d3d6d1b32c83_1.csv has 2 intersections in a4's file
Subject 03, Window OCDetect_03_recording_16_ef53c0f1-0043-446a-a484-0bb20ae8f6c5_9.csv has 2 intersections in a1's file
Subject 03, Window OCDetect_03_recording_08_f08133be-a401-412e-bc35-d9a04d0d7744_9.csv has 2 intersections in a2's file
Subject 03, Window OCDetect_03_recording_09_ca585847-6ab5-4121-a1bf-87ca67bf0dfa_9.csv has 2 intersections in a2's file
Subject 18, Window OCDetect_18_recording_11_96c2f6f5-546e-467b-bc93-9932dc050cdb_5.csv has 2 intersections in a1's file
Subject 18, Window OCDetect_18_recording_12_5d4248c2-3e00-45b4-8509-752615967f45_8.csv has 2 intersections in a1's file
Subject 18, Window OCDetect_18_recording_16_2f7b283a-62ea-4aab-aa30-ff0d79abdb01_2.csv has 2 intersections in a1's file
Subject 18, Window OCDetect_18_recording_19_86578a1f-740e-4a62-b6f0-43ba758b678f_2.csv has 2 intersections in a1's file
Subject 18, Window OCDetect_18_recording

In [6]:
#create data set with automated labels durations per subject
out_path = os.path.join(main_dir, thesis, "automated_labels")
if not os.path.exists(out_path):
    os.makedirs(out_path)
automated_labels_path = "/dhc/groups/ocdetect/preprocessed_relabeled"

for subject in subjects: 
    subject_out_path = os.path.join(out_path, f"subject_{subject}.csv")
    with open(subject_out_path, 'w', newline='') as csvfile:
        fieldnames = ['filename', 'start', 'end']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        print(subject)
        start_time = 0
        end_time = 0
        durations = []
        for filename in os.listdir(automated_labels_path):
            file_path = os.path.join(automated_labels_path, filename)
        
            # Check if the path points to a file
            if os.path.isfile(file_path) and f"OCDetect_{subject}" in file_path:
                print(file_path)
                df = pd.read_csv(file_path)
                start_index = file_path.find("OCDetect")
                filename = file_path[start_index:]
                for ind, row in df.iterrows():
                    if row["relabeled"] != 0 and start_time == 0:
                        start_time = pd.to_datetime(row['datetime'], format='%Y-%m-%d %H:%M:%S.%f')
                    if row["relabeled"] == 0 and start_time != 0:
                        end_time = pd.to_datetime(row['datetime'], format='%Y-%m-%d %H:%M:%S.%f') - pd.Timedelta(seconds=0.02)
                        duration = (end_time - start_time).total_seconds()
                        durations.append(duration)
                        writer.writerow({'filename': filename, 'start': start_time.strftime('%Y-%m-%d %H:%M:%S.%f'), 'end': end_time.strftime('%Y-%m-%d %H:%M:%S.%f')})
                        start_time = 0
                        end_time = 0
        print("subject ", subject)
        print(durations)

03
/dhc/groups/ocdetect/preprocessed_relabeled/OCDetect_03_recording_22_97be822d-5df9-4790-9a7d-5dbac9a2f3f4.csv
/dhc/groups/ocdetect/preprocessed_relabeled/OCDetect_03_recording_20_203bdef8-3281-46b8-bd73-4940e7e5e1f7.csv
/dhc/groups/ocdetect/preprocessed_relabeled/OCDetect_03_recording_13_4b27c776-7013-4adb-8710-0fedc5b5104a.csv
/dhc/groups/ocdetect/preprocessed_relabeled/OCDetect_03_recording_02_1e3760c3-ffc1-4ab3-ae03-9a0f1ae18f11.csv
/dhc/groups/ocdetect/preprocessed_relabeled/OCDetect_03_recording_03_43c50dfb-dded-4aa3-821b-1cd6df93b946.csv
/dhc/groups/ocdetect/preprocessed_relabeled/OCDetect_03_recording_01_922c33df-5363-49ec-890f-f9da9785cc76.csv
/dhc/groups/ocdetect/preprocessed_relabeled/OCDetect_03_recording_04_0a48395d-614f-497c-ab71-0d579d74ce27.csv
/dhc/groups/ocdetect/preprocessed_relabeled/OCDetect_03_recording_05_382535ec-9a0d-4359-b120-47f7605a22de.csv
/dhc/groups/ocdetect/preprocessed_relabeled/OCDetect_03_recording_16_ef53c0f1-0043-446a-a484-0bb20ae8f6c5.csv
/dhc/gr